# Visualizing AOP Hyperspectral Data in Google Earth Engine (GEE) using the Python API
Authors: John Musinsky, Bridget Hass

Modified from Qiusheng Wu's [GEE Tutorial #9 - Interactive plotting of Earth Engine data with minimal coding](https://www.youtube.com/watch?v=PDab8mkAFL0), [giswqs_geemap_plotting_notebook](https://github.com/giswqs/geemap/blob/master/examples/notebooks/09_plotting.ipynb)

## AOP data in GEE

[Google Earth Engine](https://earthengine.google.com/) is a platform idea for carrying out continental and planetary scale geospatial analyses. It has a multi-pedabyte catalog of satellite imagery and geospatial datasets, and is a powerful tool for comparing and scaling up NEON airborne datasets. 



AOP has published a subset of AOP raster (L3) data products for a handful of NEON sites on GEE. To interactively explore NEON data available on GEE, you can use the [aop-data-visualization](https://neon-aop.users.earthengine.app/view/aop-data-visualization) app created by John Musinsky. 

The table below shows the sites, products, and years of data that can currently be accessed in GEE. The * indicates partial availability.

| Domain/Site | Years      | Data Products           |
|----------|------------|-------------------------|
| D08 TALL | 2017, 2018 | SDR, RGB, CHM, DSM, DTM |"
| D11 CLBJ | 2017, 2019 | SDR, RGB, CHM, DSM, DTM |
| D14 JORN | 2017, 2019 | SDR, RGB (2019 only), DSM, DTM      |
| D14 SRER | 2017, 2018, 2019, 2021* | SDR, RGB, CHM*, DSM, DTM      |
| D16 WREF | 2017, 2018 | SDR, RGB, CHM, DSM, DTM |
| D17 TEAK | 2017, 2018 | SDR, RGB, CHM, DSM, DTM |



The NEON data products can be accessed through the `projects/neon` folder with an appended prefix of the data product ID (DPID) similar to what you see on the [NEON data portal](https://data.neonscience.org/data-products/explore). The table below shows the corresponding prefixes to use for given data products.

| Acronym | Data Product      | Data Product ID (Prefix)          |
|----------|------------|-------------------------|
| SDR | Surface Directional Reflectance | DP3-30006-001_SDR |
| RGB | Red Green Blue (Camera Imagery) | DP3-30010-001_RGB |
| CHM | Canopy Height Model | DP3-30015-001_CHM |
| DSM | Digital Surface Model | DP3-30024-001_DSM |
| DTM | Digital Terrain Model | DP3-30024-001_DTM |

## SRER Hyperspectral Visualization
In this tutorial, we will visualize the Santa Rita (SRER) SDR (hyperspectral) data collected from 2017 to 2021. The data product prefix to pull in the data is (`projects/neon/DP3-30006-001_SDR`). 

This tutorial uses the `geemap` Python package, and was modified from the Jupyter notebook [GEE Tutorial #9 - Interactive plotting of Earth Engine data with minimal coding](https://geemap.org/notebooks/09_plotting/). Instead of using Landsat data, we will pull in AOP data.

To access the NEON AOP data you can either use the earth engine `ImageCollection`
`SRER_SDR2018 = ee.Image("projects/neon/D14_SRER/L3/DP3-30006-001_D14_SRER_SDR_2018")`
`ee.ImageCollection("projects/neon/DP3-30006-001_SDR").filterDate('2018-01-01', '2018-12-31').filterBounds(geo).first();`

First, import the relevant Earth Engine (ee) packages, [ee](https://developers.google.com/earth-engine/guides/python_install) and [geemap](https://geemap.org/).

In [ ]:
import ee, geemap
%pprint

Next you will need to generate a code to Authenticate and then Initialize before getting started.

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
Map = geemap.Map()
Map

## Add SRER Surface Directional Reflectance Data

In [ ]:
# Specify center location of SRER
geo = ee.Geometry.Point([-110.83549, 31.91068])

# Read in Surface Directional Reflectance (SDR) Images 
SRER_SDR2018 = ee.Image("projects/neon/D14_SRER/L3/DP3-30006-001_D14_SRER_SDR_2018")
SRER_SDR2019 = ee.Image("projects/neon/D14_SRER/L3/DP3-30006-001_D14_SRER_SDR_2019")
SRER_SDR2021 = ee.Image("projects/neon/D14_SRER/L3/DP3-30006-001_D14_SRER_SDR_2021")

# Mask layers to only show values > 0 (this hides the no data values of -9999) 
SRER_SDR2018mask = SRER_SDR2018.updateMask(SRER_SDR2018.gte(0.0000))
SRER_SDR2019mask = SRER_SDR2019.updateMask(SRER_SDR2019.gte(0.0000))
SRER_SDR2021mask = SRER_SDR2021.updateMask(SRER_SDR2021.gte(0.0000))

# Set the visualization parameters so contrast is maximized, and set display to show RGB bands 
visParams = {'min':2,'max':20,'gamma':0.9,'bands':['band053','band035','band019']};

# Add the 3 years of SRER SDR data as layers to the Map:
Map.addLayer(SRER_SDR2018mask, visParams, 'SRER 2018');
Map.addLayer(SRER_SDR2019mask, visParams, 'SRER 2019');
Map.addLayer(SRER_SDR2021mask, visParams, 'SRER 2021');

# Center the map on SRER
Map.setCenter(-110.83549, 31.91068, 11);

TIP: If you don't know the exact image name, you can use the ImageCollection function and filter by date and bounds, as follows (eg, for 2018 SRER SDR):

`SRER_SDR2018 = ee.ImageCollection("projects/neon/DP3-30006-001_SDR").filterDate('2018-01-01', '2018-12-31').filterBounds(geo).first();`

In [ ]:
props = geemap.image_props(SRER_SDR2021)
props.getInfo()

In [ ]:
bands = SRER_SDR2021.get('system:band_names').getInfo()
bands

##  Water Vapor "Bad" Band Windows 
We can see from the spectral profiles that there are anomalously high and noisy reflectance values in two areas. These result from water vapor which absorbs light between wavelengths 1340-1445 nm and 1790-1955 nm. In addition some of the highest wavelengths tend to have a lot of noise. The atmospheric correction that converts radiance to reflectance subsequently results in a spike at these band ranges. The wavelengths of these water vapor bands is stored in the h5 reflectance attributes, but for this exercise we will remove the bands between 190-212, 280-314, and 416-427, which correspond to the noisy regions. We'll use list comprehension to make a list of all the bad bands, using the `band###` format corresponding to the band labels.

In [ ]:
bad_bands = ['band' + str(a) for a in range(190,212)] + ['band' + str(a) for a in range(280,314)] + ['band' + str(a) for a in range(416,427)]
bad_bands

The bands that we want to use, which we'll call `good_bands` are all of the bands except these bad bands:

In [ ]:
good_bands = [b for b in bands if b not in bad_bands]
good_bands

In [ ]:
print('# of all bands:',len(bands))
print('# of good bands:',len(good_bands))

We can see that we've considerably reduced the number of bands, but these are the bands that we'll want to use when doing our analysis.

In [ ]:
SRER_SDR2021sub = SRER_SDR2021.select(good_bands)

In [ ]:
visParams2 = {'min':2,'max':20,'gamma':0.9,'bands':['band053','band035','band019']};
SRER_SDR2021sub_mask = SRER_SDR2021sub.updateMask(SRER_SDR2021sub.gte(0.0000))
Map.addLayer(SRER_SDR2021sub_mask, visParams2, 'SRER 2021 Good Bands')
Map

### Map.set_plot_options 

There are various options to change with the plot, for this example we will set overlay to true. Refer to the [documentation](https://geemap.org/geemap/#geemap.geemap.Map.set_plot_options) for more options.

In [ ]:
Map.set_plot_options?

In [ ]:
Map.set_plot_options(add_marker_cluster=True)

In [ ]:
Map.set_plot_options(overlay=True)

In [ ]:
# https://developers.google.com/earth-engine/apidocs/ee-image-rename
band_names = SRER_SDR2021.get('system:band_names').getInfo()
bands_renamed = [str(int(band.replace('band',''))) for band in band_names]
bands_renamed[:5]

## Optional Exercises

Here are a few more exercises to try out:
1. Subsetting by every 10th band
2. Explore the GEE map functionality to convert Javascript to Python.

In [ ]:
good_bands_sub10 = good_bands[::10]
good_bands_sub10
# good_wl_sub20_rename = [b.replace('band','') for b in good_bands_sub20]

Select only the "good" bands and only select every 10 bands:

In [ ]:
SRER_SDR2021sub10 = SRER_SDR2021.select(good_bands_sub10)
SRER_SDR2021sub10_mask = SRER_SDR2021sub10.updateMask(SRER_SDR2021sub10.gte(0.0000))
visParams = {'min':2,'max':20,'gamma':0.9,'bands':['band051','band031','band021']};
Map.addLayer(SRER_SDR2021sub10_mask, visParams, 'SRER 2021 Good Bands Subset 10')

### geemap.js_snippet_to_py
`js_snippet_to_py` converts a snippet of `JavaScript` (js) code to python. Try out on your own! 

In [ ]:
geemap.js_snippet_to_py?

In [ ]:
js_snippet = ""
geemap.js_snippet_to_py(js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True)

## Additional Python-GEE Resources to Explore!
- https://developers.google.com/earth-engine/guides/python_install
- https://github.com/giswqs
- https://github.com/giswqs/geemap/blob/master/examples/notebooks/
- https://github.com/giswqs/earthengine-py-notebooks
- https://www.youtube.com/playlist?list=PLAxJ4-o7ZoPccOFv1dCwvGI6TYnirRTg3
- https://geemap.org/workshops/GeoPython_2021/
- https://courses.spatialthoughts.com/end-to-end-gee.html
- https://earthlab.colorado.edu/introduction-google-earth-engine-python-api

Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305